### i202473 Muhammad Huzaifa - i200474 Ahsan Rasheed
#### CS - E
#### AI Part 2

##### importing libraries

In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


##### reading dataset

In [3]:
dp1 = pd.read_csv('dataset.csv')

##### cleaning data

In [4]:
dp1.drop_duplicates(inplace=True)
dp1.dropna(thresh=len(dp1.columns), inplace=True)

In [5]:
#dropping useless rows
dp1 = dp1.query("has_round_started == True")
dp1 = dp1.query("Player2_health < 200")
dp1 = dp1.query("Player1_health < 200")
dp1.reset_index(inplace=True)
dp1 = dp1.drop('index',axis=1)

##### feature engineering and preprocessing

In [6]:
#Finding euclidean distance between p1 and p2
dp1['EuDistance'] = ((dp1['Player1_x_coord'] - dp1['Player2_x_coord'])**2 + (dp1['Player1_y_coord'] - dp1['Player2_y_coord'])**2)**0.5
dp1 = dp1.drop(['Player1_is_player_in_move','fight_result', 'Player1_x_coord', 'Player2_x_coord','Player1_y_coord', 'Player2_y_coord', 'has_round_started', 'Player2_health', 'Player1_health', 'Player2_Y', 'Player2_B', 'Player2_X', 'Player2_L', 'Player2_R', 'Player2_A', 'Player1_health', 'Player2_health', 'Player1_is_jumping', 'Player1_is_crouching', 'Player2_up', 'Player2_down','Player2_right', 'Player2_left'], axis=1)

In [7]:
#renaming columns for convenience
dp1 = dp1.rename(columns={'Player2_ID': 'ID2','Player1_ID': 'ID1', 'Player1_up': 'up', 'Player1_down': 'down', 'Player1_right': 'right', 'Player1_left': 'left', 'Player1_Y': 'Y', 'Player1_B': 'B'
                          , 'Player1_X': 'X', 'Player1_A': 'A', 'Player1_L': 'L', 'Player1_R': 'R'})


In [8]:
#converting bools to integers
dp1[['up', 'down', 'left', 'right', 'Y', 'B', 'X', 'A', 'L', 'R', 'Player2_is_jumping','Player2_is_crouching', 'Player2_is_player_in_move']]=dp1[['up', 'down', 'left', 'right', 'Y', 'B', 'X', 'A', 'L', 'R', 'Player2_is_jumping','Player2_is_crouching', 'Player2_is_player_in_move']].astype(int)

In [9]:
dp1

,ID1,Player1_move_id,up,down,right,left,Y,B,X,A,L,R,ID2,Player2_is_jumping,Player2_is_crouching,Player2_is_player_in_move,Player2_move_id,EuDistance
0,0,0,0,0,1,0,1,0,1,0,1,0,4,0,0,0,0,102.0
1,0,0,0,0,0,0,1,1,1,0,0,0,4,0,0,0,0,102.0
2,0,0,0,0,0,1,1,1,1,0,1,0,4,0,0,0,0,102.0
3,0,0,0,1,1,1,0,1,1,0,0,0,4,0,0,1,131073,104.0
4,0,0,0,0,1,0,0,0,0,0,0,1,4,0,0,1,131073,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20122,8,1,0,0,0,0,1,0,0,0,0,0,6,0,0,0,0,72.0
20123,8,1,0,0,0,0,1,1,1,1,0,0,6,0,0,0,0,70.0
20124,8,1,1,0,1,0,1,1,0,1,1,0,6,0,1,1,33686532,70.0
20125,8,1,0,0,0,0,0,1,1,0,0,0,6,0,1,1,33686532,70.0


##### creating and training model

In [10]:
y = dp1[['up', 'down', 'right', 'left', 'Y', 'B', 'X', 'A', 'L', 'R']]
x = dp1[['ID1', 'ID2','Player2_is_jumping', 'Player2_is_crouching','Player2_is_player_in_move', 'EuDistance','Player1_move_id','Player2_move_id']]

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2)

In [12]:
rf = RandomForestClassifier(n_estimators=200, max_depth=12)

In [13]:
rf.fit(X_train, Y_train)

RandomForestClassifier(max_depth=12, n_estimators=200)

##### evaluating model

In [14]:
Y_pred = rf.predict(X_test)

In [16]:
accuracy = accuracy_score(Y_test, Y_pred)
print('Score: {:.2f}%'.format(accuracy*100))

Score: 44.73%


In [17]:
results = pd.DataFrame({'ID2': X_test['ID2'], 'ID1': X_test['ID1'], 'Player2_is_jumping': X_test['Player2_is_jumping'], 'Player2_is_crouching': X_test['Player2_is_crouching'], 'Player2_is_player_in_move': X_test['Player2_is_player_in_move'], 'Distance' : X_test['EuDistance'],
                        'up': Y_pred[:,0], 'down': Y_pred[:,1], 'right': Y_pred[:,2], 'left': Y_pred[:,3], 'Y': Y_pred[:,4], 'B': Y_pred[:,5], 'X': Y_pred[:,6], 'A': Y_pred[:,7], 'L': Y_pred[:,8], 'R': Y_pred[:,9]})


In [18]:
results

,ID2,ID1,Player2_is_jumping,Player2_is_crouching,Player2_is_player_in_move,Distance,up,down,right,left,Y,B,X,A,L,R
5808,5,1,1,0,1,134.825814,0,0,0,0,1,1,0,0,0,1
17211,6,7,0,0,1,147.054412,0,0,1,0,1,1,0,0,0,1
16900,6,7,1,0,0,177.045192,0,0,1,0,1,1,0,0,0,0
15123,6,5,0,0,0,61.000000,0,0,1,0,1,1,0,0,0,0
19200,6,8,0,0,0,29.410882,0,0,1,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10165,6,2,0,1,0,57.000000,0,0,1,0,1,1,0,0,1,0
12110,6,3,1,0,0,91.438504,0,0,0,0,1,1,0,0,0,0
4435,5,2,1,0,0,38.052595,0,0,0,0,1,1,0,0,0,0
8765,6,1,0,0,0,112.000000,0,0,1,0,1,1,0,0,0,0


##### exporting model

In [19]:
with open('botModel.pkl', 'wb') as f:
    pickle.dump(rf, f)